# Emulsion Steady Shear (Herschel-Bulkley)

Fit multiple emulsion volume fractions with Herschel–Bulkley to mirror pyRheo rotation demos.

**Data:** examples/data/pyRheo/emulsion/*.csv


In [1]:
# Google Colab compatibility - uncomment if running in Colab
# !pip install -q rheojax
# from google.colab import drive
# drive.mount('/content/drive')


## Setup and Imports

In [2]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
%matplotlib inline

import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from rheojax.core.data import RheoData
from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.fractional_maxwell_model import FractionalMaxwellModel
from rheojax.models.generalized_maxwell import GeneralizedMaxwell
from rheojax.models.herschel_bulkley import HerschelBulkley
from rheojax.pipeline.base import Pipeline
from rheojax.transforms.mastercurve import Mastercurve

jax, jnp = safe_import_jax()
verify_float64()
np.set_printoptions(precision=4, suppress=True)
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
warnings.filterwarnings('ignore', category=RuntimeWarning)

def r2_complex(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    ss_res = np.sum(np.abs(y_true - y_pred) ** 2)
    ss_tot = np.sum(np.abs(y_true - np.mean(y_true)) ** 2)
    return float(1 - ss_res / ss_tot)

def mpe(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return float(np.mean(np.abs(y_true - y_pred) / np.maximum(np.abs(y_true), 1e-12)) * 100)


INFO:2025-12-06 04:33:08,705:jax._src.xla_bridge:808: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: dlopen(libtpu.so, 0x0001): tried: 'libtpu.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibtpu.so' (no such file), '/usr/lib/libtpu.so' (no such file, not in dyld cache), 'libtpu.so' (no such file)


Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: dlopen(libtpu.so, 0x0001): tried: 'libtpu.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibtpu.so' (no such file), '/usr/lib/libtpu.so' (no such file, not in dyld cache), 'libtpu.so' (no such file)


Loading rheojax version 0.4.0


/Users/b80985/Projects/rheojax/.venv/lib/python3.13/site-packages/piblin_jax/__init__.py:28: UserWarning: GPU acceleration is only available on Linux with CUDA 12+. Using JAX in CPU mode.
  from . import backend, data, dataio, transform


## Load steady-shear data

In [3]:
DATA_DIR = Path.cwd().parent / 'data' / 'pyRheo' / 'emulsion'
files = sorted(DATA_DIR.glob('*.csv'))
frames = []
for f in files:
    df = pd.read_csv(f)
    df.columns = df.columns.str.strip()
    df = df.dropna()
    df['phi'] = f.stem
    frames.append(df)
all_df = pd.concat(frames, ignore_index=True)
print(all_df.head())


   Shear Rate    Stress   phi  0.001003995054076478  4.6626050233384495  \
0    0.001570  4.717239  0.69                   NaN                 NaN   
1    0.002472  4.704130  0.69                   NaN                 NaN   
2    0.004041  4.743921  0.69                   NaN                 NaN   
3    0.006120  4.774533  0.69                   NaN                 NaN   
4    0.010096  4.855836  0.69                   NaN                 NaN   

   0.0009962854762886107  7.445899342994139  0.0010021527764970915  \
0                    NaN                NaN                    NaN   
1                    NaN                NaN                    NaN   
2                    NaN                NaN                    NaN   
3                    NaN                NaN                    NaN   
4                    NaN                NaN                    NaN   

   9.948157317915243  0.0010147845822195973  13.147187538756189  \
0                NaN                    NaN                 N

## Fit Herschel-Bulkley per volume fraction

In [4]:
fig, axes = plt.subplots(1, len(files), figsize=(5*len(files),5), sharey=True)
for ax, f in zip(axes, files):
    df = pd.read_csv(f)
    df.columns = df.columns.str.strip()
    df = df.dropna()
    gamma_dot = df.iloc[:,0].to_numpy()
    tau = df.iloc[:,1].to_numpy()

    model = HerschelBulkley()
    model.fit(gamma_dot, tau, test_mode='rotation')
    tau_pred = model.predict(gamma_dot)
    r2 = model.score(gamma_dot, tau)

    ax.loglog(gamma_dot, tau, 'o', label='Data')
    ax.loglog(gamma_dot, tau_pred, '-', label=f'HB fit (R^2={r2:.3f})')
    ax.set_title(f'phi={f.stem}')
    ax.set_xlabel('Shear rate (1/s)')
    ax.grid(True, which='both', ls='--', alpha=0.4)
axes[0].set_ylabel('Stress (Pa)')
axes[-1].legend()
plt.tight_layout()
plt.show()


/var/folders/rq/lgpr8fvj349cr_wvkzv99md80000gp/T/ipykernel_15945/52199356.py:22: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
